In [1]:
users_interests = [
    ["Hadoop", "Big Data", "HBase", "Java", "Spark", "Storm", "Cassandra"],
    ["NoSQL", "MongoDB", "Cassandra", "HBase", "Postgres"],
    ["Python", "scikit-learn", "scipy", "numpy", "statsmodels", "pandas"],
    ["R", "Python", "statistics", "regression", "probability"],
    ["machine learning", "regression", "decision trees", "libsvm"],
    ["Python", "R", "Java", "C++", "Haskell", "programming languages"],
    ["statistics", "probability", "mathematics", "theory"],
    ["machine learning", "scikit-learn", "Mahout", "neural networks"],
    ["neural networks", "deep learning", "Big Data", "artificial intelligence"],
    ["Hadoop", "Java", "MapReduce", "Big Data"],
    ["statistics", "R", "statsmodels"],
    ["C++", "deep learning", "artificial intelligence", "probability"],
    ["pandas", "R", "Python"],
    ["databases", "HBase", "Postgres", "MySQL", "MongoDB"],
    ["libsvm", "regression", "support vector machines"]
]

In [2]:
from collections import Counter

popular_interests = Counter(interest
                            for user_interests in users_interests
                            for interest in user_interests)

from typing import Dict, List, Tuple

def most_popular_new_interests(
        user_interests: List[str],
        max_results: int = 5) -> List[Tuple[str, int]]:
    suggestions = [(interest, frequency)
                   for interest, frequency in popular_interests.most_common()
                   if interest not in user_interests]
    return suggestions[:max_results]

unique_interests = sorted({interest
                           for user_interests in users_interests
                           for interest in user_interests})

In [3]:
unique_interests[:6]

['Big Data', 'C++', 'Cassandra', 'HBase', 'Hadoop', 'Haskell']

In [4]:
def make_user_interest_vector(user_interests: List[str]) -> List[int]:
    """
    Given a list ofinterests, produce a vector whose ith element is 1
    if unique_interests[i] is in the list, 0 otherwise
    """
    return [1 if interest in user_interests else 0
            for interest in unique_interests]

user_interest_vectors = [make_user_interest_vector(user_interests)
                         for user_interests in users_interests]

In [5]:
from linear_algebra import dot, Vector
import math

def cosine_similarity(v1: Vector, v2: Vector) -> float:
    return dot(v1, v2) / math.sqrt(dot(v1, v1) * dot(v2, v2))

assert cosine_similarity([1., 1, 1], [2., 2, 2]) == 1, "same direction"
assert cosine_similarity([-1., -1], [2., 2]) == -1,    "opposite direction"
assert cosine_similarity([1., 0], [0., 1]) == 0,       "orthogonal"

In [6]:
user_similarities = [[cosine_similarity(interest_vector_i, interest_vector_j)
                      for interest_vector_j in user_interest_vectors]
                     for interest_vector_i in user_interest_vectors]

# Users 0 and 9 share interests in Hadoop, Java, and Big Data
assert 0.56 < user_similarities[0][9] < 0.58, "several shared interests"

# Users 0 and 8 share only one interest: Big Data
assert 0.18 < user_similarities[0][8] < 0.20, "only one shared interest"

In [7]:
def most_similar_users_to(user_id: int) -> List[Tuple[int, float]]:
    pairs = [(other_user_id, similarity)                      # Find other
             for other_user_id, similarity in                 # users with
                enumerate(user_similarities[user_id])         # nonzero
             if user_id != other_user_id and similarity > 0]  # similarity.

    return sorted(pairs,                                      # Sort them
                  key=lambda pair: pair[-1],                  # most similar
                  reverse=True)                               # first.


most_similar_to_zero = most_similar_users_to(0)
user, score = most_similar_to_zero[0]
assert user == 9
assert 0.56 < score < 0.57
user, score = most_similar_to_zero[1]
assert user == 1
assert 0.33 < score < 0.34

In [8]:
from collections import defaultdict

def user_based_suggestions(user_id: int,
                           include_current_interests: bool = False):
    # Sum up the similarities.
    suggestions: Dict[str, float] = defaultdict(float)
    for other_user_id, similarity in most_similar_users_to(user_id):
        for interest in users_interests[other_user_id]:
            suggestions[interest] += similarity

    # Convert them to a sorted list.
    suggestions = sorted(suggestions.items(),
                         key=lambda pair: pair[-1],  # weight
                         reverse=True)

    # And (maybe) exclude already-interests
    if include_current_interests:
        return suggestions
    else:
        return [(suggestion, weight)
                for suggestion, weight in suggestions
                if suggestion not in users_interests[user_id]]


In [9]:
ubs0 = user_based_suggestions(0)
interest, score = ubs0[0]
assert interest == 'MapReduce'
assert 0.56 < score < 0.57
interest, score = ubs0[1]
assert interest == 'MongoDB'
assert 0.50 < score < 0.51

In [10]:
interest_user_matrix = [[user_interest_vector[j]
                         for user_interest_vector in user_interest_vectors]
                        for j, _ in enumerate(unique_interests)]

[1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]

interest_similarities = [[cosine_similarity(user_vector_i, user_vector_j)
                          for user_vector_j in interest_user_matrix]
                         for user_vector_i in interest_user_matrix]

def most_similar_interests_to(interest_id: int):
    similarities = interest_similarities[interest_id]
    pairs = [(unique_interests[other_interest_id], similarity)
             for other_interest_id, similarity in enumerate(similarities)
             if interest_id != other_interest_id and similarity > 0]
    return sorted(pairs,
                  key=lambda pair: pair[-1],
                  reverse=True)


msit0 = most_similar_interests_to(0)
assert msit0[0][0] == 'Hadoop'
assert 0.815 < msit0[0][1] < 0.817
assert msit0[1][0] == 'Java'
assert 0.666 < msit0[1][1] < 0.667

In [11]:
def item_based_suggestions(user_id: int,
                           include_current_interests: bool = False):
    # Add up the similar interests
    suggestions = defaultdict(float)
    user_interest_vector = user_interest_vectors[user_id]
    for interest_id, is_interested in enumerate(user_interest_vector):
        if is_interested == 1:
            similar_interests = most_similar_interests_to(interest_id)
            for interest, similarity in similar_interests:
                suggestions[interest] += similarity

    # Sort them by weight
    suggestions = sorted(suggestions.items(),
                         key=lambda pair: pair[-1],
                         reverse=True)

    if include_current_interests:
        return suggestions
    else:
        return [(suggestion, weight)
                for suggestion, weight in suggestions
                if suggestion not in users_interests[user_id]]

In [12]:
ibs0 = item_based_suggestions(0)
assert ibs0[0][0] == 'MapReduce'
assert 1.86 < ibs0[0][1] < 1.87
assert ibs0[1][0] in ('Postgres', 'MongoDB')  # A tie
assert 1.31 < ibs0[1][1] < 1.32


In [16]:
def main():
    
    # Replace this with the locations of your files
    
    # This points to the current directory, modify if your files are elsewhere.
    MOVIES = "movies.txt"   # pipe-delimited: movie_id|title|...
    RATINGS = "ratings.txt"  # tab-delimited: user_id, movie_id, rating, timestamp
    
    from typing import NamedTuple
    
    class Rating(NamedTuple):
        user_id: str
        movie_id: str
        rating: float
    
    import csv
    # We specify this encoding to avoid a UnicodeDecodeError.
    # see: https://stackoverflow.com/a/53136168/1076346
    with open(MOVIES, encoding="iso-8859-1") as f:
        reader = csv.reader(f, delimiter="|")
        movies = {movie_id: title for movie_id, title, *_ in reader}
    
    # Create a list of [Rating]
    with open(RATINGS, encoding="iso-8859-1") as f:
        reader = csv.reader(f, delimiter="\t")
        ratings = [Rating(user_id, movie_id, float(rating))
                   for user_id, movie_id, rating, _ in reader]
    
    # 1682 movies rated by 943 users
    assert len(movies) == 1682
    assert len(list({rating.user_id for rating in ratings})) == 943
    
    import re
    
    # Data structure for accumulating ratings by movie_id
    star_wars_ratings = {movie_id: []
                         for movie_id, title in movies.items()
                         if re.search("Star Wars|Empire Strikes|Jedi", title)}
    
    # Iterate over ratings, accumulating the Star Wars ones
    for rating in ratings:
        if rating.movie_id in star_wars_ratings:
            star_wars_ratings[rating.movie_id].append(rating.rating)
    
    # Compute the average rating for each movie
    avg_ratings = [(sum(title_ratings) / len(title_ratings), movie_id)
                   for movie_id, title_ratings in star_wars_ratings.items()]
    
    # And then print them in order
    for avg_rating, movie_id in sorted(avg_ratings, reverse=True):
        print(f"{avg_rating:.2f} {movies[movie_id]}")
    
    import random
    random.seed(0)
    random.shuffle(ratings)
    
    split1 = int(len(ratings) * 0.7)
    split2 = int(len(ratings) * 0.85)
    
    train = ratings[:split1]              # 70% of the data
    validation = ratings[split1:split2]   # 15% of the data
    test = ratings[split2:]               # 15% of the data
    
    avg_rating = sum(rating.rating for rating in train) / len(train)
    baseline_error = sum((rating.rating - avg_rating) ** 2
                         for rating in test) / len(test)
    
    # This is what we hope to do better than
    assert 1.26 < baseline_error < 1.27
    
    
    # Embedding vectors for matrix factorization model
    
    from deep_learning import random_tensor
    
    EMBEDDING_DIM = 2
    
    # Find unique ids
    user_ids = {rating.user_id for rating in ratings}
    movie_ids = {rating.movie_id for rating in ratings}
    
    # Then create a random vector per id
    user_vectors = {user_id: random_tensor(EMBEDDING_DIM)
                    for user_id in user_ids}
    movie_vectors = {movie_id: random_tensor(EMBEDDING_DIM)
                     for movie_id in movie_ids}
    
    
    # Training loop for matrix factorization model
    
    from typing import List
    import tqdm
    from linear_algebra import dot
    
    def loop(dataset: List[Rating],
             learning_rate: float = None) -> None:
        with tqdm.tqdm(dataset) as t:
            loss = 0.0
            for i, rating in enumerate(t):
                movie_vector = movie_vectors[rating.movie_id]
                user_vector = user_vectors[rating.user_id]
                predicted = dot(user_vector, movie_vector)
                error = predicted - rating.rating
                loss += error ** 2
    
                if learning_rate is not None:
                    #     predicted = m_0 * u_0 + ... + m_k * u_k
                    # So each u_j enters output with coefficent m_j
                    # and each m_j enters output with coefficient u_j
                    user_gradient = [error * m_j for m_j in movie_vector]
                    movie_gradient = [error * u_j for u_j in user_vector]
    
                    # Take gradient steps
                    for j in range(EMBEDDING_DIM):
                        user_vector[j] -= learning_rate * user_gradient[j]
                        movie_vector[j] -= learning_rate * movie_gradient[j]
    
                t.set_description(f"avg loss: {loss / (i + 1)}")
    
    learning_rate = 0.05
    for epoch in range(20):
        learning_rate *= 0.9
        print(epoch, learning_rate)
        loop(train, learning_rate=learning_rate)
        loop(validation)
    loop(test)

if __name__ == "__main__": main()

4.36 Star Wars (1977)
4.20 Empire Strikes Back, The (1980)
4.01 Return of the Jedi (1983)
0 0.045000000000000005


avg loss: 14.605495773180305:  14%|██████▍                                      | 9931/70000 [00:07<00:43, 1366.64it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

avg loss: 1.006464302560768:   0%|                                                 | 99/70000 [00:00<01:10, 989.98it/s]

2 0.03645000000000001


avg loss: 1.0779734539470005:   5%|██▏                                           | 3422/70000 [00:03<01:11, 936.51it/s]


KeyboardInterrupt: 

In [1]:
MOVIES = "movies.txt"   # pipe-delimited: movie_id|title|...
RATINGS = "ratings.txt"  # tab-delimited: user_id, movie_id, rating, timestamp
    
from typing import NamedTuple
    
class Rating(NamedTuple):
    user_id: str
    movie_id: str
    rating: float
    
import csv
    # We specify this encoding to avoid a UnicodeDecodeError.
    # see: https://stackoverflow.com/a/53136168/1076346
with open(MOVIES, encoding="iso-8859-1") as f:
    reader = csv.reader(f, delimiter="|")
    movies = {movie_id: title for movie_id, title, *_ in reader}
    
    # Create a list of [Rating]
with open(RATINGS, encoding="iso-8859-1") as f:
    reader = csv.reader(f, delimiter="\t")
    ratings = [Rating(user_id, movie_id, float(rating))
                for user_id, movie_id, rating, _ in reader]

In [2]:
movies

{'1': 'Toy Story (1995)',
 '2': 'GoldenEye (1995)',
 '3': 'Four Rooms (1995)',
 '4': 'Get Shorty (1995)',
 '5': 'Copycat (1995)',
 '6': 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 '7': 'Twelve Monkeys (1995)',
 '8': 'Babe (1995)',
 '9': 'Dead Man Walking (1995)',
 '10': 'Richard III (1995)',
 '11': 'Seven (Se7en) (1995)',
 '12': 'Usual Suspects, The (1995)',
 '13': 'Mighty Aphrodite (1995)',
 '14': 'Postino, Il (1994)',
 '15': "Mr. Holland's Opus (1995)",
 '16': 'French Twist (Gazon maudit) (1995)',
 '17': 'From Dusk Till Dawn (1996)',
 '18': 'White Balloon, The (1995)',
 '19': "Antonia's Line (1995)",
 '20': 'Angels and Insects (1995)',
 '21': 'Muppet Treasure Island (1996)',
 '22': 'Braveheart (1995)',
 '23': 'Taxi Driver (1976)',
 '24': 'Rumble in the Bronx (1995)',
 '25': 'Birdcage, The (1996)',
 '26': 'Brothers McMullen, The (1995)',
 '27': 'Bad Boys (1995)',
 '28': 'Apollo 13 (1995)',
 '29': 'Batman Forever (1995)',
 '30': 'Belle de jour (1967)',
 '31': 'Crimson Tide

In [3]:
ratings

[Rating(user_id='196', movie_id='242', rating=3.0),
 Rating(user_id='186', movie_id='302', rating=3.0),
 Rating(user_id='22', movie_id='377', rating=1.0),
 Rating(user_id='244', movie_id='51', rating=2.0),
 Rating(user_id='166', movie_id='346', rating=1.0),
 Rating(user_id='298', movie_id='474', rating=4.0),
 Rating(user_id='115', movie_id='265', rating=2.0),
 Rating(user_id='253', movie_id='465', rating=5.0),
 Rating(user_id='305', movie_id='451', rating=3.0),
 Rating(user_id='6', movie_id='86', rating=3.0),
 Rating(user_id='62', movie_id='257', rating=2.0),
 Rating(user_id='286', movie_id='1014', rating=5.0),
 Rating(user_id='200', movie_id='222', rating=5.0),
 Rating(user_id='210', movie_id='40', rating=3.0),
 Rating(user_id='224', movie_id='29', rating=3.0),
 Rating(user_id='303', movie_id='785', rating=3.0),
 Rating(user_id='122', movie_id='387', rating=5.0),
 Rating(user_id='194', movie_id='274', rating=2.0),
 Rating(user_id='291', movie_id='1042', rating=4.0),
 Rating(user_id='2

In [23]:
import re
    
    # Data structure for accumulating ratings by movie_id
star_wars_ratings = {movie_id: [] for movie_id, title in movies.items() 
                     if re.search("Star Wars|Empire Strikes|Jedi", title)}
    
    # Iterate over ratings, accumulating the Star Wars ones
for rating in ratings:
    if rating.movie_id in star_wars_ratings:
        star_wars_ratings[rating.movie_id].append(rating.rating)
    
    # Compute the average rating for each movie
avg_ratings = [(sum(title_ratings) / len(title_ratings), movie_id) 
               for movie_id, title_ratings in star_wars_ratings.items()]
    
    # And then print them in order
for avg_rating, movie_id in sorted(avg_ratings, reverse=True):
    print(f"{avg_rating:.2f} {movies[movie_id]}")
    

4.36 Star Wars (1977)
4.20 Empire Strikes Back, The (1980)
4.01 Return of the Jedi (1983)


In [24]:
import random
random.seed(0)
random.shuffle(ratings)
    
split1 = int(len(ratings) * 0.7)
split2 = int(len(ratings) * 0.85)
    
train = ratings[:split1]              # 70% of the data
validation = ratings[split1:split2]   # 15% of the data
test = ratings[split2:]               # 15% of the data
    
avg_rating = sum(rating.rating for rating in train) / len(train)
baseline_error = sum((rating.rating - avg_rating) ** 2 for rating in test) / len(test)

In [25]:
baseline_error

1.2609526646939684

In [26]:
avg_rating

3.530842857142857

In [28]:
from deep_learning import random_tensor
    
EMBEDDING_DIM = 2
    
    # Find unique ids
user_ids = {rating.user_id for rating in ratings}
movie_ids = {rating.movie_id for rating in ratings}
    
    # Then create a random vector per id
user_vectors = {user_id: random_tensor(EMBEDDING_DIM) for user_id in user_ids}
movie_vectors = {movie_id: random_tensor(EMBEDDING_DIM) for movie_id in movie_ids}
    
    

In [35]:
movie_vectors

{'1507': [1.6083431243896484, 0.5195331573486328],
 '826': [-0.08355140686035156, 0.16198158264160156],
 '1069': [-0.4845714569091797, -1.1136341094970703],
 '1524': [0.3532695770263672, 0.013933181762695312],
 '354': [0.1683521270751953, 0.3493785858154297],
 '870': [-0.7375049591064453, 0.3389263153076172],
 '959': [1.7627429962158203, 0.5962276458740234],
 '584': [-1.7546749114990234, -0.2257061004638672],
 '471': [0.5320262908935547, 0.6962490081787109],
 '468': [1.750802993774414, -1.5021610260009766],
 '182': [0.029268264770507812, -0.6501674652099609],
 '1317': [-1.0232067108154297, -1.2176227569580078],
 '963': [0.4649066925048828, -0.39681434631347656],
 '1220': [-0.6098842620849609, 0.3987407684326172],
 '990': [-0.8049106597900391, -0.16564369201660156],
 '1087': [-1.4587116241455078, 1.0023212432861328],
 '757': [1.263437271118164, 2.7731990814208984],
 '1358': [-0.7033824920654297, 1.4244365692138672],
 '975': [-1.2380504608154297, -0.29738426208496094],
 '141': [2.0983791